In [1]:
import copy
import os
import random
import sys

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from PIL import Image
from sklearn.metrics import cohen_kappa_score, precision_score, recall_score, accuracy_score
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from torchvision.transforms.functional import to_pil_image
from tqdm import tqdm

from library import *

In [12]:
torch.manual_seed(0)
batch_size = 24
num_classes = 5  # 5 DR levels
learning_rate = 0.0001
num_epochs = 20

In [4]:
transform_train = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomCrop((210, 210)),
    SLORandomPad((224, 224)),
    FundRandomRotate(prob=0.5, degree=30),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(90),
    transforms.ColorJitter(brightness=(0.1, 0.9)),
    transforms.RandomInvert(p=0.7), # 0.7
    transforms.RandomGrayscale(p=0.5), # 0.5
    transforms.RandomPerspective(p=0.3), # 0.3    
    transforms.GaussianBlur(kernel_size=5), # 5
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [9]:
def train_and_save(model, name, num_epochs=10): 
    train_dataset = RetinopathyDataset('./DeepDRiD/train.csv', './DeepDRiD/train/', transform_train)
    val_dataset = RetinopathyDataset('./DeepDRiD/val.csv', './DeepDRiD/val/', transform_test)
    test_dataset = RetinopathyDataset('./DeepDRiD/test.csv', './DeepDRiD/test/', transform_test, test=True)
    
    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    # Define the weighted CrossEntropyLoss
    criterion = nn.CrossEntropyLoss()
    
    # Use GPU device is possible
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print('Device:', device)
    
    # Move class weights to the device
    model = model.to(device)
    
    # Optimizer and Learning rate scheduler
    optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate)
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    
    # Train and evaluate the model with the training and validation set
    model, kappas = train_model(
        model, train_loader, val_loader, device, criterion, optimizer,
        lr_scheduler=lr_scheduler, num_epochs=num_epochs,
        checkpoint_path='./aptos-resnet-deepdrip_{}.pth'.format(name)
    )

    # Load the pretrained checkpoint
    state_dict = torch.load('./aptos-resnet-deepdrip_{}.pth'.format(name), map_location='cpu')
    model.load_state_dict(state_dict, strict=True)
    
    # Make predictions on testing set and save the prediction results
    evaluate_model(model, test_loader, device, test_only=True, prediction_path="./test_predictions_aptos-{}-deepdrip.csv".format(name))
    return model, kappas

In [13]:
# Freeze all layers 
num_epochs = 10
model = models.resnet18()
name = "resnet"

state_dict = torch.load('./model_aptos-2019-pretrained_ResNET.pth', map_location='cpu')
model.load_state_dict(state_dict, strict=True)
for param in model.parameters():
    param.requires_grad = True

model, kappas = train_and_save(model, name)

C:\Users\a4lu9\AppData\Local\Temp\ipykernel_23892\814716308.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('./model_aptos-2019-pretrained_ResNET

Device: cuda

Epoch 1/10
Training: 100%|█████████████████████████████████████████| 100/100 [00:22<00:00,  4.37 batch/s, lr=1.0e-04, Loss=1.1243]
[Train] Kappa: 0.5446 Accuracy: 0.4829 Precision: 0.4668 Recall: 0.4829 Loss: 1.5352
[Train] Class 0: Precision: 0.6241, Recall: 0.7611
[Train] Class 1: Precision: 0.4191, Recall: 0.3937
[Train] Class 2: Precision: 0.3347, Recall: 0.3396
[Train] Class 3: Precision: 0.4677, Recall: 0.4521
[Train] Class 4: Precision: 0.3529, Recall: 0.1750
[Train] Class 5: Precision: 0.0000, Recall: 0.0000
Evaluating: 100%|██████████████████████████████████████████████████████████████████| 34/34 [00:04<00:00,  8.17 batch/s]
[Val] Kappa: 0.7687 Accuracy: 0.5975 Precision: 0.6180 Recall: 0.5975

Epoch 2/10
Training: 100%|█████████████████████████████████████████| 100/100 [00:23<00:00,  4.21 batch/s, lr=1.0e-04, Loss=1.2425]
[Train] Kappa: 0.6820 Accuracy: 0.6021 Precision: 0.5913 Recall: 0.6021 Loss: 0.9858
[Train] Class 0: Precision: 0.7363, Recall: 0.8569
[Train

C:\Users\a4lu9\AppData\Local\Temp\ipykernel_23892\2066115805.py:33: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('./aptos-resnet-deepdrip_{}.pth'.fo

Evaluating: 100%|██████████████████████████████████████████████████████████████████| 34/34 [00:07<00:00,  4.54 batch/s]
[Test] Save predictions to C:\Users\a4lu9\koulu\DeepLearning\project\521153S-3005-final-project\test_predictions_aptos-resnet-deepdrip.csv


In [14]:
# Freeze all layers 
num_epochs = 10
model = models.vgg16()
name = "vgg"

state_dict = torch.load('./model_aptos-2019-pretrained_VGG.pth', map_location='cpu')
model.load_state_dict(state_dict, strict=True)
for param in model.parameters():
    param.requires_grad = True

# Unfreeze the last layer
model, kappas = train_and_save(model, name)

C:\Users\a4lu9\AppData\Local\Temp\ipykernel_23892\2489708349.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('./model_aptos-2019-pretrained_VGG.p

Device: cuda

Epoch 1/10
Training: 100%|█████████████████████████████████████████| 100/100 [08:10<00:00,  4.90s/ batch, lr=1.0e-04, Loss=1.4473]
[Train] Kappa: 0.5893 Accuracy: 0.4642 Precision: 0.4362 Recall: 0.4642 Loss: 1.3121
[Train] Class 0: Precision: 0.6177, Recall: 0.7472
[Train] Class 1: Precision: 0.3815, Recall: 0.3354
[Train] Class 2: Precision: 0.2869, Recall: 0.2958
[Train] Class 3: Precision: 0.4725, Recall: 0.5375
[Train] Class 4: Precision: 0.2273, Recall: 0.0625
Evaluating: 100%|██████████████████████████████████████████████████████████████████| 34/34 [00:06<00:00,  5.12 batch/s]
[Val] Kappa: 0.7176 Accuracy: 0.5525 Precision: 0.5376 Recall: 0.5525

Epoch 2/10
Training: 100%|█████████████████████████████████████████| 100/100 [08:04<00:00,  4.85s/ batch, lr=1.0e-04, Loss=1.2185]
[Train] Kappa: 0.6893 Accuracy: 0.5700 Precision: 0.5509 Recall: 0.5700 Loss: 1.0717
[Train] Class 0: Precision: 0.7503, Recall: 0.8389
[Train] Class 1: Precision: 0.5052, Recall: 0.5042
[Train

C:\Users\a4lu9\AppData\Local\Temp\ipykernel_23892\2066115805.py:33: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('./aptos-resnet-deepdrip_{}.pth'.fo

Evaluating: 100%|██████████████████████████████████████████████████████████████████| 34/34 [00:08<00:00,  3.93 batch/s]
[Test] Save predictions to C:\Users\a4lu9\koulu\DeepLearning\project\521153S-3005-final-project\test_predictions_aptos-vgg-deepdrip.csv


In [15]:
# Freeze all layers 
num_epochs = 10
model = models.densenet161()
name = "densenet"

state_dict = torch.load('./model_aptos-2019-pretrained_DenseNET.pth', map_location='cpu')
model.load_state_dict(state_dict, strict=True)
for param in model.parameters():
    param.requires_grad = True

# Unfreeze the last layer
model, kappas = train_and_save(model, name)

C:\Users\a4lu9\AppData\Local\Temp\ipykernel_23892\2457236606.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('./model_aptos-2019-pretrained_Dense

Device: cuda

Epoch 1/10
Training: 100%|█████████████████████████████████████████| 100/100 [16:48<00:00, 10.09s/ batch, lr=1.0e-04, Loss=1.0817]
[Train] Kappa: 0.5806 Accuracy: 0.5112 Precision: 0.4957 Recall: 0.5112 Loss: 1.3543
[Train] Class 0: Precision: 0.6627, Recall: 0.7778
[Train] Class 1: Precision: 0.4469, Recall: 0.4208
[Train] Class 2: Precision: 0.3710, Recall: 0.3833
[Train] Class 3: Precision: 0.5063, Recall: 0.5021
[Train] Class 4: Precision: 0.3200, Recall: 0.1667
[Train] Class 5: Precision: 0.0000, Recall: 0.0000
[Train] Class 6: Precision: 0.0000, Recall: 0.0000
[Train] Class 7: Precision: 0.0000, Recall: 0.0000
[Train] Class 8: Precision: 0.0000, Recall: 0.0000
[Train] Class 9: Precision: 0.0000, Recall: 0.0000
[Train] Class 10: Precision: 0.0000, Recall: 0.0000
Evaluating: 100%|██████████████████████████████████████████████████████████████████| 34/34 [01:25<00:00,  2.51s/ batch]
[Val] Kappa: 0.8390 Accuracy: 0.6475 Precision: 0.7362 Recall: 0.6475

Epoch 2/10
Traini

C:\Users\a4lu9\AppData\Local\Temp\ipykernel_23892\2066115805.py:33: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load('./aptos-resnet-deepdrip_{}.pth'.fo

Evaluating: 100%|██████████████████████████████████████████████████████████████████| 34/34 [00:52<00:00,  1.55s/ batch]
[Test] Save predictions to C:\Users\a4lu9\koulu\DeepLearning\project\521153S-3005-final-project\test_predictions_aptos-densenet-deepdrip.csv
